<table>
    <tr>
        <td>
        <center>
        <font size="+1">If you haven't used BigQuery datasets on Kaggle previously, check out the <a href = "https://www.kaggle.com/rtatman/sql-scavenger-hunt-handbook/">Scavenger Hunt Handbook</a> kernel to get started.</font>
        </center>
        </td>
    </tr>
</table>

# SELECT, FROM & WHERE

Today, we're going to learn how to use SELECT, FROM and WHERE to get data from a specific column based on the value of another column. For the purposes of this explanation, we'll be using this imaginary database, `pet_records` which has just one table in it, called `pets`, which looks like this:

![](https://i.imgur.com/Ef4Puo3.png)

### SELECT ... FROM
___

The most basic SQL query is to select a single column from a specific table. To do this, you need to tell SELECT which column to select and then specify what table that column is from using from. 

> **Do you need to capitalize SELECT and FROM?** No, SQL doesn't care about capitalization. However, it's customary to capitalize your SQL commands and it makes your queries a bit easier to read.

So, if we wanted to select the "Name" column from the pets table of the pet_records database (if that database were accessible as a BigQuery dataset on Kaggle , which it is not, because I made it up), we would do this:

    SELECT Name
    FROM `bigquery-public-data.pet_records.pets`

Which would return the highlighted data from this figure.

![](https://i.imgur.com/8FdVyFP.png)

### WHERE ...
___

When you're working with BigQuery datasets, you're almost always going to want to return only certain rows, usually based on the value of a different column. You can do this using the WHERE clause, which will only return the rows where the WHERE clause evaluates to true.

Let's look at an example:

    SELECT Name
    FROM `bigquery-public-data.pet_records.pets`
    WHERE Animal = "Cat"

This query will only return the entries from the "Name" column that are in rows where the "Animal" column has the text "Cat" in it. Those are the cells highlighted in blue in this figure:

![](https://i.imgur.com/Va52Qdl.png)


## Example: What are all the U.S. cities in the OpenAQ dataset?
___

Now that you've got the basics down, let's work through an example with a real dataset. Today we're going to be working with the OpenAQ dataset, which has information on air quality around the world. (The data in it should be current: it's updated weekly.)

To help get you situated, I'm going to run through a complete query first. Then it will be your turn to get started running your queries!

First, I'm going to set up everything we need to run queries and take a quick peek at what tables are in our database.

In [1]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
open_aq = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="openaq")

# print all the tables in this dataset (there's only one!)
open_aq.list_tables()

['global_air_quality']

I'm going to take a peek at the first couple of rows to help me see what sort of data is in this dataset.

In [2]:
# print the first couple rows of the "global_air_quality" dataset
open_aq.head("global_air_quality")

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Mobile_Cle Elum,037,US,pm25,0.00,2017-09-26 20:00:00+00:00,µg/m³,AirNow,47.197630,-120.958230,1.0
1,Mobile_WhiteSalmon,039,US,pm25,0.00,2017-09-26 20:00:00+00:00,µg/m³,AirNow,45.732414,-121.492330,1.0
2,Mobile_Newport,051,US,pm25,0.00,2017-09-21 18:00:00+00:00,µg/m³,AirNow,48.186485,-117.049160,1.0
3,FR20047,Ain,FR,o3,2.13,2018-02-13 07:00:00+00:00,µg/m³,EEA France,45.823223,4.953958,1.0
4,FR20047,Ain,FR,no2,45.40,2018-02-13 07:00:00+00:00,µg/m³,EEA France,45.823223,4.953958,1.0


Great, everything looks good! Now that I'm set up, I'm going to put together a query. I want to select all the values from the "city" column for the rows there the "country" column is "us" (for "United States"). 

> **What's up with the triple quotation marks (""")?** These tell Python that everything inside them is a single string, even though we have line breaks in it. The line breaks aren't necessary, but they do make it much easier to read your query.

In [3]:
# query to select all the items from the "city" column where the
# "country" column is "us"
query = """SELECT city
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'US'
        """

> **Important:**  Note that the argument we pass to FROM is *not* in single or double quotation marks (' or "). It is in backticks (\`). If you use quotation marks instead of backticks, you'll get this error when you try to run the query: `Syntax error: Unexpected string literal` 

Now I can use this query to get information from our open_aq dataset. I'm using the `BigQueryHelper.query_to_pandas_safe()` method here because it won't run a query if it's larger than 1 gigabyte, which helps me avoid accidentally running a very large query. See the [Scavenger Hunt Handbook ](https://www.kaggle.com/rtatman/sql-scavenger-hunt-handbook/)for more details. 

In [4]:
# the query_to_pandas_safe will only return a result if it's less
# than one gigabyte (by default)
us_cities = open_aq.query_to_pandas_safe(query)

Now I've got a dataframe called us_cities, which I can use like I would any other dataframe:

In [5]:
# What five cities have the most measurements taken there?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     85
Houston                                     75
Los Angeles-Long Beach-Santa Ana            60
New York-Northern New Jersey-Long Island    57
Riverside-San Bernardino-Ontario            56
Name: city, dtype: int64

# Scavenger hunt
___

Now it's your turn! Here's the questions I would like you to get the data to answer:

* Which countries use a unit other than ppm to measure any type of pollution? (Hint: to get rows where the value *isn't* something, use "!=")
* Which pollutants have a value of exactly 0?

In order to answer these questions, you can fork this notebook by hitting the blue "Fork Notebook" at the very top of this page (you may have to scroll up).  "Forking" something is making a copy of it that you can edit on your own without changing the original.

Let's list the table schema before proceeding. This will help us to form the correct query.

In [6]:
open_aq.table_schema(table_name="global_air_quality")

[SchemaField('location', 'string', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'string', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'string', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'string', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'float', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'timestamp', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'string', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'string', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'float', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'float', 'NULLABLE', 'Longitude in d

In [7]:
# Your code goes here :)

#Which countries use a unit other than ppm to measure any type of pollution?
query = """SELECT country
           FROM `bigquery-public-data.openaq.global_air_quality`
           WHERE unit != "ppm"
        """

#print(open_aq.estimate_query_size(query=query))
no_ppm_countries = open_aq.query_to_pandas_safe(max_gb_scanned=0.5, query=query)
print(no_ppm_countries['country'].value_counts())

FR    2638
ES    1876
DE    1382
US    1267
AT     789
CZ     600
TR     484
CL     406
GB     403
CN     384
BE     372
IT     320
NL     278
IN     230
PT     199
BR     188
NO     154
TW     154
AU     141
SK     132
MN     125
RU     124
HU     108
FI     102
CH      97
HK      89
CA      88
BA      74
MX      70
MK      67
      ... 
LU      26
DK      26
SE      24
IE      24
GH      22
PE      21
MT      18
LV      13
RS       8
SI       8
AD       7
GI       6
AR       4
VN       3
ID       3
ZA       2
NP       2
BD       2
ET       2
SG       1
BH       1
IL       1
UG       1
NG       1
LK       1
CO       1
AE       1
KW       1
XK       1
PH       1
Name: country, Length: 64, dtype: int64


So, above are all the countries that use 'ppm' as a unit and seems like FR(France?) uses it the most!

In [8]:
#Which pollutants have a value of exactly 0?

query = """SELECT pollutant
           FROM `bigquery-public-data.openaq.global_air_quality`
           WHERE value = 0.00
        """

pollutants_with_value_0 = open_aq.query_to_pandas_safe(query=query) #default max_gb_scanned is 1
pollutants_with_value_0["pollutant"].value_counts()

so2     346
no2     118
o3      110
co      100
pm10     72
pm25     64
Name: pollutant, dtype: int64

The above 7 pollutants have a value of 0.00

Please feel free to ask any questions you have in this notebook or in the [Q&A forums](https://www.kaggle.com/questions-and-answers)! 

Also, if you want to share or get comments on your kernel, remember you need to make it public first! You can change the visibility of your kernel under the "Settings" tab, on the right half of your screen.